# VQE & QAOA Patterns (EXAM CRITICAL!)

> **Exam Weight**: Part of 12% (Section 6) | **Must Master**: ✅✅✅

## Learning Objectives
By the end of this notebook, you will be able to:
- Understand the VQE algorithm and variational principle
- Build parameterized ansatz circuits (RealAmplitudes, EfficientSU2)
- Create Hamiltonians using SparsePauliOp
- Implement the VQE optimization loop with Estimator
- Understand QAOA for combinatorial optimization

---

## 🧠 Conceptual Deep Dive

### The Valley Hiker Analogy 🏔️

Think of **VQE** as a **blindfolded hiker** trying to find the lowest point in a valley:

| Hiking | VQE Equivalent | Purpose |
|--------|---------------|---------|
| **The Valley** | Energy landscape | All possible energies for parameters θ |
| **Your Position** | Parameters θ | Current guess for optimal values |
| **Altitude Check** | Estimator ⟨H⟩ | Measure energy at current θ |
| **Compass** | Classical optimizer | Tells you which direction to step |
| **Lowest Point** | Ground state E₀ | The minimum energy (goal!) |

```
VQE: The Blindfolded Descent
┌─────────────────────────────────────────────────────┐
│                                                     │
│   Energy                                            │
│     ▲        *← Start here (random θ)               │
│     │       /                                       │
│     │      /                                        │
│     │     *  ← Measure ⟨H⟩, adjust θ                │
│     │      \                                        │
│     │       \                                       │
│     │        *  ← Getting lower...                  │
│     │         \                                     │
│     │          ★ ← Ground state E₀ (goal!)         │
│     └──────────────────────────────▶ Parameters θ  │
│                                                     │
│  Variational Principle: ⟨ψ(θ)|H|ψ(θ)⟩ ≥ E₀        │
│  → Any guess gives energy ≥ true minimum!          │
│                                                     │
└─────────────────────────────────────────────────────┘
```

### VQE Components (MEMORIZE!)
```
VQE = Ansatz + Hamiltonian + Estimator + Optimizer
      ───────   ───────────   ─────────   ─────────
      |ψ(θ)⟩    H (SparsePauliOp)  ⟨H⟩    scipy.minimize
```

> **Memory Trick**: "AHEO" - **A**nsatz, **H**amiltonian, **E**stimator, **O**ptimizer

---

## Setup

## Part 1: VQE Overview

**VQE (Variational Quantum Eigensolver)**: Hybrid algorithm to find ground state energy

```
Goal: Minimize ⟨ψ(θ)|H|ψ(θ)⟩

Process:
1. Prepare |ψ(θ)⟩ (quantum circuit with parameters)
2. Measure ⟨H⟩ using Estimator
3. Adjust θ using classical optimizer
4. Repeat until converged

Variational Principle: ⟨ψ|H|ψ⟩ ≥ E₀
→ Minimizing energy converges to ground state!
```

**Key Property**: Any trial state gives energy ≥ ground state energy

## Part 2: VQE Components Breakdown

In [ ]:
# VQE Components - All in one cell for reference

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

# 1. ANSATZ (Parameterized Circuit)
theta = Parameter('θ')
phi = Parameter('φ')

ansatz = QuantumCircuit(2)
ansatz.ry(theta, 0)
ansatz.ry(phi, 1)
ansatz.cx(0, 1)

print("Ansatz circuit:")
print(ansatz.draw())

# 2. HAMILTONIAN (Observable)
H = SparsePauliOp.from_list([
    ('ZI', 1.0),
    ('IZ', 1.0),
    ('XX', 0.5)
])
print(f"\nHamiltonian: {H}")

# 3. ESTIMATOR
estimator = StatevectorEstimator()

# 4. COST FUNCTION
def cost_function(params):
    qc = ansatz.assign_parameters(params)
    job = estimator.run([(qc, H)])
    return job.result()[0].data.evs

# 5. OPTIMIZE
initial_params = [0.0, 0.0]
result = minimize(cost_function, initial_params, method='COBYLA')

print(f"\n✓ Optimal parameters: θ={result.x[0]:.4f}, φ={result.x[1]:.4f}")
print(f"✓ Ground energy: {result.fun:.6f}")

### 🎯 EXAM CRITICAL: Cost Function Pattern

**MEMORIZE THIS EXACT PATTERN**:

```python
def cost_function(params):
    # 1. Bind parameters
    qc = ansatz.assign_parameters(params)
    
    # 2. Run Estimator
    job = estimator.run([(qc, H)])
    result = job.result()
    
    # 3. Extract energy
    energy = result[0].data.evs
    
    return energy
```

**Every VQE uses this pattern!**

## Part 3: Simple VQE - Single Qubit

In [ ]:
# Simple VQE: Find ground state of H = Z
# Expected: Ground state |1⟩, energy = -1

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import numpy as np

# 1. Ansatz - single rotation
theta = Parameter('θ')
ansatz = QuantumCircuit(1)
ansatz.ry(theta, 0)

print("Ansatz:")
print(ansatz.draw())

# 2. Hamiltonian
H = SparsePauliOp('Z')

# 3. Estimator
estimator = StatevectorEstimator()

# 4. Cost function
def cost(params):
    qc = ansatz.assign_parameters({theta: params[0]})
    job = estimator.run([(qc, H)])
    return job.result()[0].data.evs

# 5. Optimize
result = minimize(cost, x0=[np.pi/4], method='COBYLA')

print(f"\nOptimal θ: {result.x[0]:.4f} rad ({result.x[0]/np.pi:.2f}π)")
print(f"Ground energy: {result.fun:.6f}")
print(f"Expected: -1.0 (achieved when θ = π)")

# Explanation:
# Ry(θ)|0⟩ rotates from |0⟩ toward |1⟩
# At θ = π: Ry(π)|0⟩ = |1⟩
# ⟨1|Z|1⟩ = -1 (ground state energy)

### Common Optimizers

```python
from scipy.optimize import minimize

# COBYLA - Constrained Optimization BY Linear Approximation
result = minimize(cost, x0, method='COBYLA')
# ✓ Gradient-free
# ✓ Handles noise well
# ✓ Most common for VQE

# SLSQP - Sequential Least SQuares Programming
result = minimize(cost, x0, method='SLSQP')
# ✓ Faster convergence
# ✓ Uses gradients
# ✓ Good for noiseless simulation

# Nelder-Mead
result = minimize(cost, x0, method='Nelder-Mead')
# ✓ Simplex method
# ✓ Gradient-free
# ✓ Alternative to COBYLA
```

## Part 4: Two-Qubit VQE

In [ ]:
# Two-Qubit VQE: Find ground state of H = ZZ
# Note: For ZZ, ground states are |00⟩ and |11⟩ with energy +1
# Excited states are |01⟩ and |10⟩ with energy -1

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import numpy as np

# 1. Ansatz (2 parameters)
theta1 = Parameter('θ₁')
theta2 = Parameter('θ₂')

ansatz = QuantumCircuit(2)
ansatz.ry(theta1, 0)
ansatz.ry(theta2, 1)
ansatz.cx(0, 1)

print("Ansatz:")
print(ansatz.draw())

# 2. Hamiltonian
H = SparsePauliOp('ZZ')

# 3. Cost function
estimator = StatevectorEstimator()

def cost(params):
    qc = ansatz.assign_parameters({
        theta1: params[0],
        theta2: params[1]
    })
    job = estimator.run([(qc, H)])
    return job.result()[0].data.evs

# 4. Optimize
result = minimize(cost, x0=[0.0, 0.0], method='COBYLA')

print(f"\nOptimal params: θ₁={result.x[0]:.4f}, θ₂={result.x[1]:.4f}")
print(f"Ground energy: {result.fun:.6f}")

# ZZ eigenvalues:
# |00⟩ → (+1)(+1) = +1
# |01⟩ → (+1)(-1) = -1 ← Minimum!
# |10⟩ → (-1)(+1) = -1 ← Minimum!
# |11⟩ → (-1)(-1) = +1

### ⚠️ EXAM TRAP: Parameter Binding

```python
# Single parameter
theta = Parameter('θ')
qc = ansatz.assign_parameters({theta: value})
# OR
qc = ansatz.assign_parameters([value])  # List order

# Multiple parameters
theta1, theta2 = Parameter('θ₁'), Parameter('θ₂')
qc = ansatz.assign_parameters({
    theta1: value1,
    theta2: value2
})
# OR
qc = ansatz.assign_parameters([value1, value2])  # List order
```

**Prefer dict for clarity!**

## Part 5: VQE with Complex Hamiltonian

In [ ]:
# VQE with Complex Hamiltonian: H = ZI + IZ + 0.5*XX

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import numpy as np

# 1. Hamiltonian
H = SparsePauliOp.from_list([
    ('ZI', 1.0),
    ('IZ', 1.0),
    ('XX', 0.5)
])
print(f"Hamiltonian: {H}")

# 2. Richer ansatz with ParameterVector
params = ParameterVector('θ', 3)

ansatz = QuantumCircuit(2)
ansatz.ry(params[0], 0)
ansatz.ry(params[1], 1)
ansatz.cx(0, 1)
ansatz.rz(params[2], 1)

print("\nAnsatz:")
print(ansatz.draw())

# 3. Estimator and cost function
estimator = StatevectorEstimator()

def cost(param_values):
    qc = ansatz.assign_parameters(param_values)
    job = estimator.run([(qc, H)])
    return job.result()[0].data.evs

# 4. Optimize with random initial guess
np.random.seed(42)
x0 = np.random.random(3)
result = minimize(cost, x0, method='COBYLA', options={'maxiter': 200})

print(f"\nGround energy: {result.fun:.6f}")
print(f"Optimal params: {result.x}")
print(f"Iterations: {result.nfev}")

## Part 6: VQE Verification with Sampler

In [ ]:
# VQE Verification: Run VQE then verify with Sampler

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

# 1. Setup VQE
H = SparsePauliOp('Z')
theta = Parameter('θ')
ansatz = QuantumCircuit(1)
ansatz.ry(theta, 0)

estimator = StatevectorEstimator()

def cost(params):
    qc = ansatz.assign_parameters({theta: params[0]})
    return estimator.run([(qc, H)]).result()[0].data.evs

# 2. Run VQE
result = minimize(cost, x0=[0.5], method='COBYLA')
optimal_theta = result.x[0]

print(f"VQE Results:")
print(f"  Optimal θ: {optimal_theta:.4f}")
print(f"  Ground energy: {result.fun:.6f}")

# 3. Verify ground state with Sampler
optimal_qc = ansatz.assign_parameters({theta: optimal_theta})
optimal_qc.measure_all()

sampler = StatevectorSampler()
job = sampler.run([(optimal_qc,)], shots=1024)
counts = job.result()[0].data.meas.get_counts()

print(f"\nSampler Verification:")
print(f"  Counts: {counts}")
print(f"  Ground state |1⟩ should dominate (⟨Z⟩=-1)")

## Part 7: Complete VQE Pattern (MEMORIZE!)

In [ ]:
# COMPLETE VQE PATTERN - FULLY EXECUTABLE

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import numpy as np

# STEP 1: Define Hamiltonian
H = SparsePauliOp.from_list([
    ('ZI', 1.0),
    ('IZ', 1.0),
    ('XX', 0.5)
])
print("STEP 1: Hamiltonian defined")
print(f"H = {H}")

# STEP 2: Create Parameterized Ansatz
num_qubits = 2
num_params = 2
theta = [Parameter(f'θ{i}') for i in range(num_params)]

ansatz = QuantumCircuit(num_qubits)
ansatz.ry(theta[0], 0)
ansatz.ry(theta[1], 1)
ansatz.cx(0, 1)
print("\nSTEP 2: Ansatz created")
print(ansatz.draw())

# STEP 3: Create Estimator
estimator = Estimator()
print("\nSTEP 3: Estimator created")

# STEP 4: Define Cost Function
iteration_count = 0

def cost_function(params):
    global iteration_count
    iteration_count += 1
    
    # Bind parameters
    qc = ansatz.assign_parameters(params)
    
    # Compute expectation value
    job = estimator.run([(qc, H)])
    result = job.result()
    energy = result[0].data.evs
    
    if iteration_count % 10 == 0:
        print(f"  Iteration {iteration_count}: E = {energy:.6f}")
    
    return energy

print("\nSTEP 4: Cost function defined")

# STEP 5: Optimize
print("\nSTEP 5: Running optimization...")
initial_params = np.random.random(num_params) * np.pi

result = minimize(
    cost_function,
    initial_params,
    method='COBYLA',
    options={'maxiter': 100}
)

# STEP 6: Extract Results
optimal_params = result.x
ground_state_energy = result.fun

print(f"\n{'═'*50}")
print("STEP 6: RESULTS")
print(f"{'═'*50}")
print(f"Ground state energy: {ground_state_energy:.6f}")
print(f"Optimal parameters: θ0={optimal_params[0]:.4f}, θ1={optimal_params[1]:.4f}")
print(f"Total iterations: {iteration_count}")
print(f"Optimization success: {result.success}")

print(f"\n{'═'*50}")
print("EXAM CHECKLIST:")
print("☑ Define Hamiltonian (SparsePauliOp)")
print("☑ Create parameterized ansatz (Parameter)")
print("☑ Use Estimator in cost function")
print("☑ result[0].data.evs for expectation value")
print("☑ Optimize with scipy.optimize.minimize")
print("☑ Extract result.x (params) and result.fun (energy)")

## 📝 Practice Questions

### Question 1: VQE Components

**What are the four main components of VQE?**

A) Circuit, Backend, Transpiler, Optimizer  
B) Ansatz, Hamiltonian, Estimator, Optimizer  
C) Sampler, Estimator, Backend, Circuit  
D) Parameters, Gates, Measurements, Results

<details>
<summary>Answer</summary>

**B) Ansatz, Hamiltonian, Estimator, Optimizer**

```
1. ANSATZ - Parameterized circuit |ψ(θ)⟩
2. HAMILTONIAN - Observable H (SparsePauliOp)
3. ESTIMATOR - Computes ⟨ψ(θ)|H|ψ(θ)⟩
4. OPTIMIZER - Minimizes energy (scipy.optimize.minimize)
```

**Memory Aid**: "AHEO = Ansatz, Hamiltonian, Estimator, Optimizer"
</details>

---

### Question 2: Cost Function

**What does the VQE cost function return?**

A) Measurement counts  
B) Expectation value ⟨H⟩  
C) Probability distribution  
D) Optimal parameters

<details>
<summary>Answer</summary>

**B) Expectation value ⟨H⟩**

```python
def cost_function(params):
    qc = ansatz.assign_parameters(params)
    job = estimator.run([(qc, H)])
    energy = job.result()[0].data.evs  # Expectation value
    return energy  # ← Returns ⟨ψ(θ)|H|ψ(θ)⟩
```

**Key**: Cost function returns energy to MINIMIZE
</details>

---

### Question 3: Variational Principle

**What does the variational principle guarantee?**

A) VQE always finds exact ground state  
B) ⟨ψ|H|ψ⟩ ≥ E₀ (always above ground state)  
C) VQE converges in polynomial time  
D) All eigenvalues can be found

<details>
<summary>Answer</summary>

**B) ⟨ψ|H|ψ⟩ ≥ E₀ (always above ground state)**

```
Variational Principle:
⟨ψ|H|ψ⟩ ≥ E₀ for ANY state |ψ⟩

Where E₀ = ground state energy

Consequence:
- Minimizing ⟨ψ(θ)|H|ψ(θ)⟩ converges toward E₀
- VQE energy estimate is UPPER BOUND on true ground state
- Better ansatz → closer to E₀
```

**Key**: VQE NEVER goes below true ground state energy!
</details>

---

### Question 4: Common Optimizer

**Which optimizer is most common for VQE?**

A) Adam  
B) SGD  
C) COBYLA  
D) L-BFGS

<details>
<summary>Answer</summary>

**C) COBYLA**

```python
from scipy.optimize import minimize

# COBYLA = Constrained Optimization BY Linear Approximation
result = minimize(cost, x0, method='COBYLA')

Advantages:
✓ Gradient-free (works with noisy hardware)
✓ Handles constraints
✓ Robust to noise
✓ Most popular for NISQ VQE

Alternatives:
- SLSQP: Faster but needs gradients
- Nelder-Mead: Also gradient-free
```

**Key**: COBYLA is VQE default!
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **VQE Purpose**
   - Find ground state energy
   - Minimize ⟨ψ(θ)|H|ψ(θ)⟩
   - Hybrid quantum-classical algorithm

2. **Four Components**
   - Ansatz: Parameterized circuit
   - Hamiltonian: SparsePauliOp observable
   - Estimator: Computes ⟨H⟩
   - Optimizer: scipy.optimize.minimize

3. **Cost Function**
   - Bind parameters: assign_parameters()
   - Run Estimator: result[0].data.evs
   - Return energy for minimization

4. **Variational Principle**
   - ⟨ψ|H|ψ⟩ ≥ E₀ always
   - Minimization converges to ground state
   - VQE provides upper bound on E₀

### Critical Exam Facts

- ✅ **VQE = Ansatz + Hamiltonian + Estimator + Optimizer**
- ✅ Cost function returns `result[0].data.evs`
- ✅ Use `scipy.optimize.minimize` with COBYLA
- ✅ `result.fun` = ground state energy
- ✅ `result.x` = optimal parameters
- ✅ Variational principle: E(θ) ≥ E₀
- ✅ VQE for quantum chemistry (molecular energies)
- ✅ QAOA is VQE variant for optimization

### Common Traps

- ❌ Forgetting assign_parameters() → ✅ Bind before Estimator!
- ❌ Using Sampler for energy → ✅ Use Estimator!
- ❌ `.data.ev` (singular) → ✅ `.data.evs` (plural)
- ❌ Random initial params guaranteed optimal → ✅ Local minima possible

### Mnemonic

🧠 **"AHEO: Ansatz, Hamiltonian, Estimator, Optimize!"**

- **Ansatz** - Parameterized |ψ(θ)⟩
- **Hamiltonian** - Observable H
- **Estimator** - Compute ⟨H⟩
- **Optimize** - Minimize energy

**VQE workflow**: Prepare → Estimate → Optimize → Repeat!

**Next**: Section 7 (Results Processing)!

## Part 8: Complete VQE Optimization Pattern

### VQE with scipy.optimize

**EXAM CRITICAL**: Complete VQE workflow with classical optimizer!

In [ ]:
# Complete VQE pattern with scipy.optimize
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import numpy as np

print('Complete VQE Pattern:')
print('='*50)

# 1. Define Hamiltonian (Ising model)
H = SparsePauliOp(['ZZ', 'ZI', 'IZ'], [1.0, -1.0, -1.0])
print(f'Hamiltonian: H = ZZ - ZI - IZ')

# 2. Create parameterized ansatz
theta = Parameter('θ')
phi = Parameter('φ')

ansatz = QuantumCircuit(2)
ansatz.ry(theta, 0)
ansatz.ry(phi, 1)
ansatz.cx(0, 1)
ansatz.ry(theta, 0)

print(f'Ansatz parameters: θ, φ')
print(f'Circuit depth: {ansatz.depth()}')

# 3. Define cost function
estimator = Estimator()
iteration_count = [0]
energy_history = []

def cost_function(params):
    # Bind parameters to circuit
    qc = ansatz.assign_parameters(params)
    
    # Calculate expectation value
    job = estimator.run([(qc, H)])
    result = job.result()
    energy = result[0].data.evs
    
    # Track progress
    iteration_count[0] += 1
    energy_history.append(energy)
    
    if iteration_count[0] % 10 == 0:
        print(f'Iteration {iteration_count[0]}: E = {energy:.6f}')
    
    return energy

# 4. Optimize with classical optimizer
print('\nOptimizing...')
initial_params = [0.0, 0.0]

result = minimize(
    cost_function,
    initial_params,
    method='COBYLA',
    options={'maxiter': 100}
)

# 5. Results
print('\n' + '='*50)
print('VQE Results:')
print(f'Ground state energy: {result.fun:.6f}')
print(f'Optimal parameters: θ={result.x[0]:.4f}, φ={result.x[1]:.4f}')
print(f'Total iterations: {iteration_count[0]}')
print(f'Success: {result.success}')

print('\n✓ scipy.optimize.minimize for classical optimization')
print('✓ COBYLA is gradient-free (good for noisy functions)')
print('✓ Cost function returns ⟨H⟩ from Estimator')

### Optimizer Comparison

**EXAM TIP**: Different optimizers for different problems!

In [ ]:
# Compare different optimizers
from scipy.optimize import minimize
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp

# Setup
H = SparsePauliOp(['ZI', 'IZ', 'XX'], [1.0, 1.0, 0.5])
theta = Parameter('θ')
qc = QuantumCircuit(2)
qc.ry(theta, 0)
qc.ry(theta, 1)
qc.cx(0, 1)

estimator = Estimator()

def cost(params):
    job = estimator.run([(qc.assign_parameters(params), H)])
    return job.result()[0].data.evs

# Test different methods
methods = ['COBYLA', 'SLSQP', 'Nelder-Mead']
initial = [0.5]

print('Optimizer Comparison:')
print('='*50)
for method in methods:
    result = minimize(cost, initial, method=method, options={'maxiter': 50})
    print(f'{method:12s}: E = {result.fun:.6f}, θ = {result.x[0]:.4f}')

print('\n⚠️ EXAM NOTE:')
print('  COBYLA = Constrained Optimization BY Linear Approximation')
print('  SLSQP = Sequential Least SQuares Programming')
print('  Nelder-Mead = Simplex method')
print('\n✓ All are gradient-free (good for quantum)')
print('✓ COBYLA most common in Qiskit examples')

## Part 9: QAOA Pattern

### QAOA for MaxCut

**EXAM CRITICAL**: QAOA is a special case of VQE!

In [ ]:
# QAOA (Quantum Approximate Optimization Algorithm)
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import numpy as np

print('QAOA for MaxCut Problem:')
print('='*50)

# MaxCut on 3-node graph (triangle)
# Edges: (0,1), (1,2), (0,2)
print('Graph: Triangle with 3 nodes, 3 edges')
print('Goal: Maximize cuts (partition into two sets)')

# Cost Hamiltonian: sum of -0.5*(1-ZiZj) for each edge
H_cost = SparsePauliOp(
    ['ZZ I', 'Z IZ', 'I ZZ'],  # Edges (0,1), (0,2), (1,2)
    [1.0, 1.0, 1.0]
)

def qaoa_circuit(gamma, beta, p=1):
    """Create QAOA circuit with p layers"""
    qc = QuantumCircuit(3)
    
    # Initial state: equal superposition
    qc.h([0, 1, 2])
    
    for _ in range(p):
        # Cost layer (problem-specific)
        qc.rzz(2*gamma, 0, 1)  # Edge (0,1)
        qc.rzz(2*gamma, 0, 2)  # Edge (0,2)
        qc.rzz(2*gamma, 1, 2)  # Edge (1,2)
        
        # Mixer layer (standard)
        qc.rx(2*beta, 0)
        qc.rx(2*beta, 1)
        qc.rx(2*beta, 2)
    
    return qc

# QAOA optimization
estimator = Estimator()
iteration = [0]

def qaoa_cost(params):
    gamma, beta = params
    qc = qaoa_circuit(gamma, beta)
    
    job = estimator.run([(qc, H_cost)])
    result = job.result()
    energy = result[0].data.evs
    
    iteration[0] += 1
    if iteration[0] % 5 == 0:
        print(f'Iteration {iteration[0]}: E = {energy:.4f}')
    
    return energy

print('\nOptimizing QAOA...')
initial = [0.5, 0.5]  # [gamma, beta]
result = minimize(qaoa_cost, initial, method='COBYLA', options={'maxiter': 50})

print('\n' + '='*50)
print('QAOA Results:')
print(f'Optimal energy: {result.fun:.4f}')
print(f'Optimal γ = {result.x[0]:.4f}, β = {result.x[1]:.4f}')
print(f'Iterations: {iteration[0]}')

# Verify solution
final_qc = qaoa_circuit(result.x[0], result.x[1])
print(f'\nCircuit depth: {final_qc.depth()}')
print(f'Gate count: {final_qc.size()}')

print('\n✓ QAOA uses rzz gates for cost layer')
print('✓ rx gates for mixer layer')
print('✓ Parameters: γ (gamma) for cost, β (beta) for mixer')
print('✓ QAOA = VQE with specific ansatz structure')

### QAOA Key Concepts

**MEMORIZE FOR EXAM**:

```
QAOA Structure:
┌─────────────────────────────────────┐
│ 1. Initial State: |+⟩|+⟩...        │
│    (equal superposition)            │
├─────────────────────────────────────┤
│ 2. Cost Layer: e^(-iγH_cost)       │
│    - Problem-specific               │
│    - rzz gates for graph problems   │
├─────────────────────────────────────┤
│ 3. Mixer Layer: e^(-iβH_mixer)     │
│    - Standard rx gates              │
│    - Explores solution space        │
├─────────────────────────────────────┤
│ 4. Repeat p times (depth)          │
└─────────────────────────────────────┘
```

**Parameters**:
- γ (gamma): Cost layer angles
- β (beta): Mixer layer angles
- p: Number of layers (circuit depth)

**Key Gates**:
- `rzz(2*gamma, i, j)`: Two-qubit rotation for graph edges
- `rx(2*beta, i)`: Single-qubit mixer

**Typical Use Cases**:
- MaxCut problems
- Graph coloring
- Traveling salesman
- Portfolio optimization

## Part 10: Exam Practice Problems

### Practice Problem 1: H2 Molecule VQE

**Complete workflow**: Find ground state of H2 molecule

In [ ]:
# Practice Problem 1: H2 Molecule Ground State
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

print('Practice Problem 1: H2 Molecule VQE')
print('='*50)

# H2 Hamiltonian (from exam README)
H = SparsePauliOp(
    ["II", "ZI", "IZ", "ZZ", "XX"],
    [-1.05, 0.39, 0.39, -0.01, 0.18]
)
print('H2 Hamiltonian: -1.05*II + 0.39*ZI + 0.39*IZ - 0.01*ZZ + 0.18*XX')

# Ansatz: Simple parameterized circuit
theta = Parameter('θ')
ansatz = QuantumCircuit(2)
ansatz.ry(theta, 0)
ansatz.ry(theta, 1)
ansatz.cx(0, 1)
ansatz.ry(theta, 0)
ansatz.ry(theta, 1)

# VQE optimization
estimator = Estimator()

def h2_cost(params):
    qc = ansatz.assign_parameters(params)
    job = estimator.run([(qc, H)])
    return job.result()[0].data.evs

print('\nOptimizing...')
result = minimize(h2_cost, [0.0], method='COBYLA')

print('\nResults:')
print(f'Ground state energy: {result.fun:.6f} Hartree')
print(f'Optimal θ: {result.x[0]:.4f}')
print(f'Exact ground state: ~-1.85 Hartree')
print(f'Error: {abs(result.fun + 1.85):.6f}')

print('\n✓ Real molecular Hamiltonian')
print('✓ VQE finds approximate ground state')
print('✓ More parameters = better approximation')

### Practice Problem 2: Multi-Layer VQE Comparison

**Compare circuit depth** impact on accuracy

In [ ]:
# Practice Problem 2: Circuit Depth Comparison
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
import numpy as np

print('Practice Problem 2: VQE Depth Comparison')
print('='*50)

# Hamiltonian
H = SparsePauliOp(['ZZ', 'XX', 'YY'], [1.0, 0.5, 0.5])

def create_ansatz(n_layers):
    """Create ansatz with specified layers"""
    params = ParameterVector('θ', n_layers * 2)
    qc = QuantumCircuit(2)
    
    for i in range(n_layers):
        qc.ry(params[2*i], 0)
        qc.ry(params[2*i+1], 1)
        qc.cx(0, 1)
    
    return qc, params

estimator = Estimator()
results = []

# Test different depths
for n_layers in [1, 2, 3, 4]:
    ansatz, params = create_ansatz(n_layers)
    
    def cost(param_values):
        qc = ansatz.assign_parameters(param_values)
        job = estimator.run([(qc, H)])
        return job.result()[0].data.evs
    
    initial = np.zeros(n_layers * 2)
    result = minimize(cost, initial, method='COBYLA', options={'maxiter': 100})
    
    results.append({
        'layers': n_layers,
        'depth': ansatz.depth(),
        'energy': result.fun,
        'params': len(initial)
    })
    
    print(f'Layers={n_layers}, Depth={ansatz.depth()}, '
          f'Params={len(initial)}, E={result.fun:.6f}')

best = min(results, key=lambda x: x['energy'])
print(f'\nBest: {best["layers"]} layers with E={best["energy"]:.6f}')

print('\n✓ More layers = more expressivity')
print('✓ But also more parameters to optimize')
print('✓ Trade-off: accuracy vs optimization difficulty')

## Part 11: Exam Quick Reference

### VQE Cheat Sheet

**🎯 MEMORIZE FOR EXAM**:

```python
# Complete VQE Pattern
from scipy.optimize import minimize
from qiskit.primitives import StatevectorEstimator as Estimator

# 1. Hamiltonian
H = SparsePauliOp(['ZZ', 'XX'], [1.0, 0.5])

# 2. Ansatz
theta = Parameter('θ')
ansatz = QuantumCircuit(2)
ansatz.ry(theta, 0)
ansatz.cx(0, 1)

# 3. Cost function
estimator = Estimator()
def cost(params):
    qc = ansatz.assign_parameters(params)
    job = estimator.run([(qc, H)])
    return job.result()[0].data.evs

# 4. Optimize
result = minimize(cost, [0.0], method='COBYLA')
ground_energy = result.fun
```

**QAOA Pattern**:
```python
# Cost layer (problem-specific)
qc.rzz(2*gamma, i, j)  # For each edge

# Mixer layer (standard)
qc.rx(2*beta, i)  # For each qubit
```

### Critical Exam Points

**VQE Components** (MEMORIZE):
1. **Ansatz**: Parameterized quantum circuit |ψ(θ)⟩
2. **Hamiltonian**: Observable H (SparsePauliOp)
3. **Estimator**: Computes ⟨ψ(θ)|H|ψ(θ)⟩
4. **Optimizer**: scipy.optimize.minimize (COBYLA)

**Common Optimizers**:
- `COBYLA`: Gradient-free, constrained
- `SLSQP`: Gradient-free, sequential
- `Nelder-Mead`: Simplex method

**QAOA Specifics**:
- Initial state: |+⟩ⁿ (all qubits in superposition)
- Cost layer: `rzz(2*gamma, i, j)` for edges
- Mixer layer: `rx(2*beta, i)` for vertices
- Parameters: 2p values (γ and β for each layer)

**Key Exam Patterns**:
- H2 molecule: 5-term Hamiltonian
- Ising model: ZZ interactions
- MaxCut: Triangle graph
- Multi-layer comparison

**Memory Aids**:
- "VQE = Variational + Quantum + Eigensolver"
- "QAOA = VQE for optimization problems"
- "Cost function returns energy from Estimator"
- "Variational principle: ⟨H⟩ ≥ E₀ always"